In [2]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter,DayLocator
from matplotlib.font_manager import FontProperties

import scipy as sp
import gdal,ogr
from gdalconst import * 
import datetime
import math
import operator
import re
import os
import os.path
import requests, time
import urllib, json
import csv
import seaborn as sns

from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale
from sklearn import decomposition
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from pylab import *
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from pprint import pprint

from scipy.spatial import KDTree
from numpy import linalg as LA
from calendar import monthrange
from sklearn import metrics
import tarfile

import geopandas as gp
from shapely.geometry import Point

from dask import dataframe as dd 
import dask
import glob
import netCDF4
from netCDF4 import Dataset

import fiona
%matplotlib inline

/home/s132732/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


***
## Load Tweet + NOAA dataframe 
***

In [2]:
ill_df=pd.read_csv('data/US_2013_integrated.csv',encoding='utf-8') 
YEAR=2013
#ill_df['timestamp']=pd.to_datetime(ill_df['timestamp'], errors='coerce')
#ill_df.sort_values(['timestamp' , 'LAT', 'LON'], ascending=[True, True, True], inplace=True)

In [3]:
ill_df.head()

,Unnamed: 0,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,HUM,CC,WIND,POP,month
0,0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.71,1.91,0.00,66.990005,63.699951,4.442998,3870.3170,1
1,1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.91,2.30,0.00,66.990005,63.699951,4.442998,2680.2053,1
2,2,1,42.023269,28,-88.245266,2013-01-01,"[must, say, we, had, a, great, time, saying, p...",True,0.12,-13.60,0.00,62.250000,5.399902,4.986554,1476.1556,1
3,3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.71,1.91,0.00,66.990005,63.699951,4.442998,3870.3170,1
4,4,1,39.595677,20,-89.578959,2013-01-01,"[lastnight, i, met, a, girl, who, got, increas...",True,1.40,-7.52,4.39,87.240005,56.899902,5.610321,1190.8944,1


***
## Load USA County Shapefile
***

In [4]:
poly  = gp.GeoDataFrame.from_file("data/UScounties/UScounties.shp")
poly

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283127277658 48.546679319076, ..."
1,Ferry,Washington,53,019,53019,POLYGON ((-118.8516288013387 47.94956368481996...
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.438831576286 48.04411548512263,..."
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.972093862835 47.93915200536639,..."
4,Pend Oreille,Washington,53,051,53051,POLYGON ((-117.4385804303028 48.99991850672649...
5,Boundary,Idaho,16,021,16021,POLYGON ((-117.0291117188672 48.83807527955628...
6,Lincoln,Montana,30,053,30053,POLYGON ((-116.0554974842291 48.20848379658699...
7,Flathead,Montana,30,029,30029,POLYGON ((-113.4736300572054 47.59757632110833...
8,Glacier,Montana,30,035,30035,POLYGON ((-112.1827335389939 48.47116514063714...
9,Toole,Montana,30,101,30101,POLYGON ((-111.4223100737647 48.21776252169813...


***
## Get Centroid of all counties
***

In [5]:
centroidseries = poly['geometry'].centroid
centroids_df=pd.DataFrame({'FIPS': poly['FIPS'], 'CENTROID': centroidseries})
centroids_df.head()

,CENTROID,FIPS
0,POINT (-94.90359451079368 48.771708924392),27077
1,POINT (-118.5183032788082 48.46948437572507),53019
2,POINT (-117.8564115791511 48.39580864680615),53065
3,POINT (-119.7406103214426 48.54830685737878),53047
4,POINT (-117.2750688149767 48.53270730712843),53051


In [6]:
all_points=[Point(lon,lat) for lon, lat in zip(ill_df.LON,ill_df.LAT)]


***
## Get 7 emotion scores of each observation
***

In [7]:
##### ADD SENTIMENTIAL PIECES INTO DATAFRAME ##################################
listing = glob.glob(os.path.join('./', '2013_*.out'))
files=sorted(listing, key=lambda name: int(name.split('_')[1]))
df_from_each_file = (pd.read_csv(f,
                 sep=",", #or delim_whitespace=True, #separator is whitespace
                 header=None, #no header
                 names=['JOY','FEAR','ANGER','SADNESS','DISGUST','SHAME','GUILT'] #set columns names joy, fear, anger, sadness, disgust, shame, guilt
                 )  for f in files)
emoji_df   = pd.concat(df_from_each_file, ignore_index=True)
ill_df = pd.concat([ill_df, emoji_df],axis=1)
ill_df.head()

,Unnamed: 0,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,...,WIND,POP,month,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT
0,0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.71,1.91,...,4.442998,3870.3170,1,0.058981,0.124937,0.046433,0.056929,0.039310,0.267973,0.405437
1,1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.91,2.30,...,4.442998,2680.2053,1,0.318294,0.081209,0.095020,0.059575,0.087509,0.259487,0.098906
2,2,1,42.023269,28,-88.245266,2013-01-01,"[must, say, we, had, a, great, time, saying, p...",True,0.12,-13.60,...,4.986554,1476.1556,1,0.409712,0.091944,0.120421,0.049077,0.071539,0.148865,0.108442
3,3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.71,1.91,...,4.442998,3870.3170,1,0.609639,0.038057,0.051051,0.030337,0.068157,0.150984,0.051775
4,4,1,39.595677,20,-89.578959,2013-01-01,"[lastnight, i, met, a, girl, who, got, increas...",True,1.40,-7.52,...,5.610321,1190.8944,1,0.232204,0.036115,0.047113,0.022468,0.282932,0.315178,0.063990


In [8]:
ill_gdf = gp.GeoDataFrame(ill_df, geometry=all_points)
ill_gdf.crs = {'init' :'epsg:4326'}
ill_gdf.head()

,Unnamed: 0,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,...,POP,month,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT,geometry
0,0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.71,1.91,...,3870.3170,1,0.058981,0.124937,0.046433,0.056929,0.039310,0.267973,0.405437,POINT (-117.8850396 34.1210122)
1,1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.91,2.30,...,2680.2053,1,0.318294,0.081209,0.095020,0.059575,0.087509,0.259487,0.098906,POINT (-117.9081129 34.04699779999999)
2,2,1,42.023269,28,-88.245266,2013-01-01,"[must, say, we, had, a, great, time, saying, p...",True,0.12,-13.60,...,1476.1556,1,0.409712,0.091944,0.120421,0.049077,0.071539,0.148865,0.108442,POINT (-88.24526572000001 42.02326852)
3,3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.71,1.91,...,3870.3170,1,0.609639,0.038057,0.051051,0.030337,0.068157,0.150984,0.051775,POINT (-117.8843725 34.1214246)
4,4,1,39.595677,20,-89.578959,2013-01-01,"[lastnight, i, met, a, girl, who, got, increas...",True,1.40,-7.52,...,1190.8944,1,0.232204,0.036115,0.047113,0.022468,0.282932,0.315178,0.063990,POINT (-89.57895908 39.59567733)


***
## Map each observation to county
***

In [9]:
obser_and_county = gp.sjoin(ill_gdf, poly, op='within')
obser_and_county.head()

,Unnamed: 0,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,...,DISGUST,SHAME,GUILT,geometry,index_right,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS
0,0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.710000,1.91,...,0.039310,0.267973,0.405437,POINT (-117.8850396 34.1210122),2384,Los Angeles,California,06,037,06037
1,1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.910000,2.30,...,0.087509,0.259487,0.098906,POINT (-117.9081129 34.04699779999999),2384,Los Angeles,California,06,037,06037
3,3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.710000,1.91,...,0.068157,0.150984,0.051775,POINT (-117.8843725 34.1214246),2384,Los Angeles,California,06,037,06037
15,15,1,33.944236,4,-118.198043,2013-01-01,"[thats, bloody, sick, ]",True,15.210000,3.71,...,0.550773,0.067090,0.082424,POINT (-118.19804333 33.9442361),2384,Los Angeles,California,06,037,06037
47,47,1,34.123201,5,-117.888406,2013-01-02,"[@anthonydiamond, thanks, love, -, lt33]",True,16.940001,3.41,...,0.088186,0.121261,0.099233,POINT (-117.888406 34.1232013),2384,Los Angeles,California,06,037,06037


In [10]:
obser_and_county.dropna(axis=0, how='any',inplace=True)
county_urban_df=obser_and_county[obser_and_county['URBAN']==True]
county_rural_df=obser_and_county[obser_and_county['URBAN']==False]

***
## Compute monthly average of each county
***

In [11]:
county_urban_monthlygrouped_df=county_urban_df.groupby([county_urban_df.month,county_urban_df.FIPS])['count'].sum().to_frame().reset_index()
county_rural_monthlygrouped_df=county_rural_df.groupby([county_rural_df.month,county_rural_df.FIPS])['count'].sum().to_frame().reset_index()

county_urban_monthlygrouped_df=county_urban_monthlygrouped_df.rename(index=str, columns={"count": "monthly_cell_count_urban"})
county_rural_monthlygrouped_df=county_rural_monthlygrouped_df.rename(index=str, columns={"count": "monthly_cell_count_rural"})

county_urban_monthlygrouped_df['monthly_cell_daily_count'] = county_urban_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_urban'] / monthrange(YEAR,int(row.month))[1], axis=1)
county_rural_monthlygrouped_df['monthly_cell_daily_count'] = county_rural_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_rural'] / monthrange(YEAR,int(row.month))[1], axis=1)

county_merge_monthlygrouped_df=pd.merge(county_urban_monthlygrouped_df, county_rural_monthlygrouped_df, how='outer', on=['month','FIPS'], sort=True,
         suffixes=('_urban', '_rural'))

county_merge_monthlygrouped_df.fillna(0).head() ##MONTHLY DAILY CELL COUNT

,month,FIPS,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,1,01013,0.0,0.000000,4.0,0.129032
1,1,01069,2.0,0.064516,0.0,0.000000
2,1,01073,2.0,0.064516,0.0,0.000000
3,1,04013,6.0,0.193548,0.0,0.000000
4,1,05007,1.0,0.032258,0.0,0.000000


***
## Compute daily features of each county
***

In [12]:
county_urban_dailygrouped_df=county_urban_df.groupby([county_urban_df.timestamp,county_urban_df.FIPS]).agg({'count':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
county_rural_dailygrouped_df=county_rural_df.groupby([county_rural_df.timestamp,county_rural_df.FIPS]).agg({'count':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
#ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

county_merge_dailygrouped_df=pd.merge(county_urban_dailygrouped_df, county_rural_dailygrouped_df, how='outer', on=['timestamp','FIPS','month'], sort=True, suffixes=('_urban', '_rural'))

county_merge_daily_df=pd.merge(county_merge_dailygrouped_df,county_merge_monthlygrouped_df, how="left", on=['FIPS','month'])

county_merge_daily_df=county_merge_daily_df

#f, ax = plt.subplots(1, 1, sharey=True)
#f.set_size_inches(18.5, 10.5, forward=True)

#test=ill_merge_daily_df[ill_merge_daily_df['timestamp']=='2016-02-21']
#plt.scatter(test['LON_cut'], test['LAT_cut'], s=700, c=test['PRCP_rural'], cmap="Reds", alpha=0.5,edgecolors=None,linewidths=0)
county_merge_daily_df.head()

,timestamp,FIPS,TMIN_urban,CC_urban,month,SADNESS_urban,GUILT_urban,DISGUST_urban,FEAR_urban,count_urban,...,TMAX_rural,SHAME_rural,HUM_rural,ANGER_rural,PRCP_rural,WIND_rural,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,2013-01-01,01013,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,15.8,0.163565,91.990005,0.224248,0.0,3.431441,NaN,NaN,4.0,0.129032
1,2013-01-01,06029,2.2100,62.000000,1,0.069858,0.110384,0.186495,0.106455,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.064516,NaN,NaN
2,2013-01-01,06037,2.4575,63.699951,1,0.051741,0.159636,0.186437,0.089024,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,149.0,4.806452,NaN,NaN
3,2013-01-01,12031,4.4200,47.899902,1,0.117408,0.104279,0.122613,0.084818,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.129032,NaN,NaN
4,2013-01-01,12095,6.2700,72.500000,1,0.053592,0.143544,0.101983,0.095768,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,40.0,1.290323,NaN,NaN


In [13]:
county_merge_daily_df=county_merge_daily_df.rename(index=str, columns={"count_urban": "count_daily_urban","count_rural":"count_daily_rural"})
county_merge_daily_df['ncount_daily_urban']=county_merge_daily_df['count_daily_urban']- county_merge_daily_df['monthly_cell_daily_count_urban']
county_merge_daily_df['ncount_daily_rural']=county_merge_daily_df['count_daily_rural']- county_merge_daily_df['monthly_cell_daily_count_rural']

In [14]:
county_merge_daily_df= county_merge_daily_df.merge(centroids_df, on='FIPS', how='inner')

In [15]:
county_merge_daily_df["CENTROID_LON"] = county_merge_daily_df.CENTROID.map(lambda p: p.x)
county_merge_daily_df["CENTROID_LAT"] = county_merge_daily_df.CENTROID.map(lambda p: p.y)


In [16]:
ill_count_daily_df=pd.read_csv('data/US_2013_county.csv',encoding='utf-8',index_col=0)
print ill_count_daily_df.shape
print county_merge_daily_df.shape

cols_to_use = county_merge_daily_df.columns.difference(ill_count_daily_df.columns)
cols_to_use = cols_to_use.tolist()+ ['timestamp', 'FIPS']
print cols_to_use
county_merge_daily_df['FIPS']=county_merge_daily_df['FIPS'].astype(int)
ill_emo_count_daily_df = pd.merge(county_merge_daily_df[cols_to_use], ill_count_daily_df, how='inner', on=['timestamp', 'FIPS'],validate="one_to_one")
print ill_emo_count_daily_df.shape

(485720, 32)
(485720, 40)
['ANGER_rural', 'ANGER_urban', 'DISGUST_rural', 'DISGUST_urban', 'FEAR_rural', 'FEAR_urban', 'GUILT_rural', 'GUILT_urban', 'JOY_rural', 'JOY_urban', 'SADNESS_rural', 'SADNESS_urban', 'SHAME_rural', 'SHAME_urban', 'timestamp', 'FIPS']
(485720, 46)


In [17]:
print ill_emo_count_daily_df.columns
print "##################################################"
print "Data Shape - ", ill_emo_count_daily_df.shape
ill_emo_count_daily_df.to_csv('data/US_'+str(YEAR)+'_EMO_county.csv',encoding='utf-8')

Index([                   u'ANGER_rural',                    u'ANGER_urban',
                        u'DISGUST_rural',                  u'DISGUST_urban',
                           u'FEAR_rural',                     u'FEAR_urban',
                          u'GUILT_rural',                    u'GUILT_urban',
                            u'JOY_rural',                      u'JOY_urban',
                        u'SADNESS_rural',                  u'SADNESS_urban',
                          u'SHAME_rural',                    u'SHAME_urban',
                            u'timestamp',                           u'FIPS',
                    u'count_daily_urban',                     u'TMIN_urban',
                             u'CC_urban',                      u'HUM_urban',
                           u'TMAX_urban',                     u'PRCP_urban',
                           u'WIND_urban',                          u'month',
                    u'count_daily_rural',                     u'TMIN_rural',